In [0]:
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize #transform json data into pd df
import json

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [0]:
 url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [0]:
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data,'html.parser')

In [0]:
S_no = 0
d = dict()

table = soup.find('table')
table_rows = table.find_all('tr')
for tr in table_rows:
  td = tr.find_all('td')
  row = [i.text for i in td]
  #print(row)
  S_no +=1
  d[S_no] = row

In [37]:
d[4]

['M3A\n', 'North York\n', 'Parkwoods\n']

In [0]:
df = pd.DataFrame.from_dict(d,orient='index',columns=['Postal Code','Borough','Neighborhood'])

In [8]:
df.head()

,Postal Code,Borough,Neighborhood
1,None,None,None
2,M1A\n,Not assigned\n,\n
3,M2A\n,Not assigned\n,\n
4,M3A\n,North York\n,Parkwoods\n
5,M4A\n,North York\n,Victoria Village\n


In [0]:
df = df.replace('\n','', regex=True)

In [11]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,None,None,None
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Regent Park / Harbourfront
4,M6A,North York,Lawrence Manor / Lawrence Heights


In [15]:
df.shape

(104, 3)

In [31]:
df.drop(df.index[0], inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [32]:
df.shape

(103, 3)

In [46]:
type(df.iloc[0,1])

str

In [41]:
df['latitude'] = None
df['longitude'] = None
df.head()

,Postal Code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,Regent Park / Harbourfront,None,None
3,M6A,North York,Lawrence Manor / Lawrence Heights,None,None
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,None,None


In [0]:
for i in range(len(df)):
  address = df.iloc[i,1]+' , Toronto, Ontario'
  geolocator = Nominatim(user_agent="ny_explorer")
  location = geolocator.geocode(address)
  latitude = location.latitude
  longitude = location.longitude
  df.iloc[i,3] = latitude
  df.iloc[i,4] = longitude

In [50]:
df.head()

,Postal Code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7543,-79.4491
1,M4A,North York,Victoria Village,43.7543,-79.4491
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.6563,-79.3809
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.7543,-79.4491
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6563,-79.3809


In [82]:
#Different method of getting coordinates
'''import sys
!{sys.executable} -m pip install geocoder
import geocoder
from geopy.geocoders import arcgis as arcgis'''

'import sys\n!{sys.executable} -m pip install geocoder\nimport geocoder\nfrom geopy.geocoders import arcgis as arcgis'

In [0]:
'''code = 'M5A'
g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
lat_lng_coords = g.latlng
lat_lng_coords'''

In [83]:
df.shape

(103, 5)